# Training Item Response Theory (IRT) models

In this notebook, we show how to train your own Item Response Theory (IRT) models.

## Preparing data

Loading packages

In [1]:
import numpy as np
import pickle
from tqdm import tqdm
from irt import *
from utils import *

random_state = 42

The leaderboard dataset we will use is composed by six scenarios (sub-datasets):
1. TruthfulQA
1. GSM8K
1. Winogrande
1. ARC
1. HellaSwag
1. MMLU

MMLU is further divided into sub-scenarios (e.g., abstract algebra, anatomy, etc). Let's check scenarios and sub-scenarios:

In [2]:
scenarios

{'harness_truthfulqa_mc_0': ['harness_truthfulqa_mc_0'],
 'gsm8k': ['harness_gsm8k_5'],
 'winogrande': ['harness_winogrande_5'],
 'arc': ['harness_arc_challenge_25'],
 'hellaswag': ['harness_hellaswag_10'],
 'mmlu': ['harness_hendrycksTest_abstract_algebra_5',
  'harness_hendrycksTest_anatomy_5',
  'harness_hendrycksTest_astronomy_5',
  'harness_hendrycksTest_business_ethics_5',
  'harness_hendrycksTest_clinical_knowledge_5',
  'harness_hendrycksTest_college_biology_5',
  'harness_hendrycksTest_college_chemistry_5',
  'harness_hendrycksTest_college_computer_science_5',
  'harness_hendrycksTest_college_mathematics_5',
  'harness_hendrycksTest_college_medicine_5',
  'harness_hendrycksTest_college_physics_5',
  'harness_hendrycksTest_computer_security_5',
  'harness_hendrycksTest_conceptual_physics_5',
  'harness_hendrycksTest_econometrics_5',
  'harness_hendrycksTest_electrical_engineering_5',
  'harness_hendrycksTest_elementary_mathematics_5',
  'harness_hendrycksTest_formal_logic_5',
 

In [3]:
SELECTED_SCENARIOS = ['gsm8k', 'arc', 'hellaswag', 'harness_truthfulqa_mc_0']

# select gsm8k, arc, hellaswag
lb_scenarios = {'lb': []}
for scenario in scenarios.keys():
    if scenario in SELECTED_SCENARIOS:
        lb_scenarios['lb'].append(*scenarios[scenario])

lb_scenarios['gsm8k'] = ['harness_gsm8k_5']
lb_scenarios['arc'] = ['harness_arc_challenge_25']
lb_scenarios['hellaswag'] = ['harness_hellaswag_10']
lb_scenarios['truthfulqa'] = ['harness_truthfulqa_mc_0']

In [4]:
lb_scenarios

{'lb': ['harness_truthfulqa_mc_0',
  'harness_gsm8k_5',
  'harness_arc_challenge_25',
  'harness_hellaswag_10'],
 'gsm8k': ['harness_gsm8k_5'],
 'arc': ['harness_arc_challenge_25'],
 'hellaswag': ['harness_hellaswag_10'],
 'truthfulqa': ['harness_truthfulqa_mc_0']}

Loading leaderboard data:

In [5]:
with open('data/lb_scenarios.pickle', 'rb') as handle:
    data = pickle.load(handle)
    data['models'] = data['models'][0]

In this dataset, we have data from 395 models. Let's see the names of some of them below

In [6]:
len(data['models']), data['models'][:10]

(393,
 ['open-llm-leaderboard/details_moreh__MoMo-70B-lora-1.8.6-DPO',
  'open-llm-leaderboard/details_cloudyu__Yi-34Bx3-MoE-90B',
  'open-llm-leaderboard/details_Weyaxi__Helion-4x34B',
  'open-llm-leaderboard/details_Weyaxi__Bagel-Hermes-34B-Slerp',
  'open-llm-leaderboard/details_Weyaxi__Bagel-Hermes-2x34b',
  'open-llm-leaderboard/details_nfaheem__Marcoroni-7b-DPO-Merge',
  'open-llm-leaderboard/details_jondurbin__bagel-dpo-34b-v0.2',
  'open-llm-leaderboard/details_udkai__Turdus',
  'open-llm-leaderboard/details_gagan3012__MetaModel_moe',
  'open-llm-leaderboard/details_jeonsworld__CarbonVillain-en-10.7B-v3'])

Below, we will process the data so all correctness scores (for all scenarios) are stored in $Y$. The dictionaries `scenarios_position` and `subscenarios_position` give the position of scenarios/subscenarios correctness scores in $Y$.

In [7]:
scenarios_position, subscenarios_position = prepare_data(lb_scenarios, data)
Y = create_responses(lb_scenarios, data)
Y.shape

(393, 26700)

In [8]:
subscenarios_position['lb'].keys()

dict_keys(['harness_truthfulqa_mc_0', 'harness_gsm8k_5', 'harness_arc_challenge_25', 'harness_hellaswag_10'])

In [9]:
# fill nan values with 0
Y[np.isnan(Y)] = 0

# print stats of Y
print('Y stats:')
print('min:', np.min(Y))
print('max:', np.max(Y))
print('mean:', np.mean(Y))
print('std:', np.std(Y))

Y stats:
min: 0.0
max: 1.0000000000000002
mean: 0.7231160348021259
std: 0.44352004300399595


For example, below you can see the scores for MMLU:

In [10]:
Y[:,scenarios_position['lb']], Y[:,scenarios_position['lb']].shape

(array([[0.99999999, 0.99994655, 1.        , ..., 1.        , 0.        ,
         1.        ],
        [0.99984026, 0.99999999, 0.99908489, ..., 1.        , 1.        ,
         1.        ],
        [0.99937792, 0.99999997, 0.99732544, ..., 1.        , 1.        ,
         1.        ],
        ...,
        [0.59677787, 0.99801392, 0.20934594, ..., 0.        , 0.        ,
         0.        ],
        [0.67288482, 0.99814252, 0.3865014 , ..., 0.        , 1.        ,
         1.        ],
        [0.42345796, 0.99926741, 0.82057698, ..., 0.        , 0.        ,
         0.        ]]),
 (393, 13350))

For scenarios that have multiple subscenarios, it is usually the case that we want to give equal importance to individual subscenarios when computing the aggregated performance in that scenario. This is equivalent to using a weighted average when computing the aggregated performance. We will create balance_weights, a vector of weights to help us compute those weighted averages. These weights will be different than one only for MMLU, which is the only scenario with multiple subscenarios.

We will use this when choosing the IRT dimension.

In [11]:
scenarios_position.keys()

dict_keys(['lb', 'gsm8k', 'arc', 'hellaswag', 'truthfulqa'])

In [12]:
balance_weights = np.ones(Y.shape[1])

bench = 'lb'

selected_subscenarios = lb_scenarios[bench]

N = len(scenarios_position[bench])
n_sub = len(selected_subscenarios)
for sub in selected_subscenarios:
    n_i = len(subscenarios_position[bench][sub])
    balance_weights[subscenarios_position[bench][sub]] = N/(n_sub*n_i)  

In [13]:
subscenarios_position

{'lb': {'harness_truthfulqa_mc_0': [0,
   1,
   2,
   3,
   4,
   5,
   6,
   7,
   8,
   9,
   10,
   11,
   12,
   13,
   14,
   15,
   16,
   17,
   18,
   19,
   20,
   21,
   22,
   23,
   24,
   25,
   26,
   27,
   28,
   29,
   30,
   31,
   32,
   33,
   34,
   35,
   36,
   37,
   38,
   39,
   40,
   41,
   42,
   43,
   44,
   45,
   46,
   47,
   48,
   49,
   50,
   51,
   52,
   53,
   54,
   55,
   56,
   57,
   58,
   59,
   60,
   61,
   62,
   63,
   64,
   65,
   66,
   67,
   68,
   69,
   70,
   71,
   72,
   73,
   74,
   75,
   76,
   77,
   78,
   79,
   80,
   81,
   82,
   83,
   84,
   85,
   86,
   87,
   88,
   89,
   90,
   91,
   92,
   93,
   94,
   95,
   96,
   97,
   98,
   99,
   100,
   101,
   102,
   103,
   104,
   105,
   106,
   107,
   108,
   109,
   110,
   111,
   112,
   113,
   114,
   115,
   116,
   117,
   118,
   119,
   120,
   121,
   122,
   123,
   124,
   125,
   126,
   127,
   128,
   129,
   130,
   131,
   132,
   133,
   13

We can see below that first averaging within subscenarios and then computing a simple average is equivalent to using a weighted average from the beginning:

In [14]:
accs1 = np.mean([Y[:,subscenarios_position[bench][sub]].mean(axis=1) for sub in lb_scenarios[bench]], axis=0)
accs2 = (balance_weights*Y)[:,scenarios_position[bench]].mean(axis=1)

np.abs(accs1 - accs2).mean()

1.1356762293373007e-14

## Training IRT

Let's split the data in train and test (recent models are placed in the test set):

In [15]:
Y_test = Y[:100]
Y_train = Y[100:]

To train the IRT model, we first need to binarize the values in $Y$ because correctness is not binary for TruthfulQA.

In [16]:
Y_bin_train = np.zeros(Y_train.shape)
Y_bin_test = np.zeros(Y_test.shape)

cs = np.linspace(0.01,.99,100)  # Threshold values to consider
for scenario in lb_scenarios.keys():
    ind = scenarios_position[scenario]
    # Find the best threshold value that minimizes the difference between averages
    c = cs[np.argmin([np.mean((np.abs((Y_train[:,ind]>c).mean(axis=1)-Y_train[:,ind].mean(axis=1)))) for c in tqdm(cs)])]
    # Apply the threshold to train and test responses
    Y_bin_train[:,ind] = (Y_train[:,ind]>c).astype(int)
    Y_bin_test[:,ind] = (Y_test[:,ind]>c).astype(int)

100%|██████████| 100/100 [00:00<00:00, 2600.65it/s]


In [17]:
lb_scenarios.keys()

dict_keys(['lb', 'gsm8k', 'arc', 'hellaswag', 'truthfulqa'])

Choosing the dimension for the IRT model based on a simple validation heuristic:

In [18]:
def validate_IRT_dimension(Y_bin_train, Y_train, scenarios, Ds, device, epochs, lr):
    val_ind = list(range(0,Y_bin_train.shape[0],5)) # Validation indices
    train_ind = [i for i in range(Y_bin_train.shape[0]) if i not in val_ind]

    # Saving the training dataset in the needed format
    create_irt_dataset(Y_bin_train[train_ind], 'data/irt_val_dataset.jsonlines')

    # Trying different Ds
    errors = []  
    errors2 = []

    for D in tqdm(Ds):
        dataset_name = 'data/irt_val_dataset.jsonlines'
        model_name = 'data/irt_val_model/'
        
        # Load trained IRT model parameters
        train_irt_model(dataset_name, model_name, D, lr, epochs, device)
        A, B, Theta = load_irt_parameters(model_name)
        
        # Determine seen and unseen items for validation
        seen_items = list(range(0, Y_bin_train.shape[1], 2))
        unseen_items = list(range(1, Y_bin_train.shape[1], 2))

        # Estimate ability parameters for the validation set
        thetas = [estimate_ability_parameters(Y_bin_train[val_ind][j][seen_items], A[:, :, seen_items], B[:, :, seen_items]) for j in range(len(val_ind))]

        # Compute validation errors for each scenario and update the errors list (in the end, we give the same weight for all scenarios)
        errors2.append([])
        for scenario in scenarios.keys():
            balance_weights = np.ones(Y.shape[1])

            selected_subscenarios = scenarios[scenario]

            N = len(scenarios_position)
            n_sub = len(selected_subscenarios)
            for sub in selected_subscenarios:
                n_i = len(subscenarios_position[scenario][sub])
                balance_weights[subscenarios_position[scenario][sub]] = N/(n_sub*n_i) 
            
            ind = [u for u in unseen_items if u in scenarios_position[scenario]]
            errors2[-1].append(np.mean([abs((balance_weights*item_curve(thetas[j], A, B))[0,ind].mean()-Y_train[val_ind][j,ind].mean())for j in range(len(val_ind))]))
        errors.append(np.mean(errors2[-1]))

    return errors, errors2, Ds

def train_IRT_for_scenario(Y_bin_train, Y_train, scenario, all_scenarios):
    Ds = [2,]#5,10,15] # Dimensions to try TODO: uncomment the rest of the dimensions
    device = 'cuda' # Either 'cuda' or 'cpu' 
    epochs = 2000  # Number of epochs for IRT model training (py-irt default is 2000)
    lr = .1  # Learning rate for IRT model training (py-irt default is .1)

    errors, errors2, Ds = validate_IRT_dimension(Y_bin_train, Y_train, {scenario: all_scenarios[scenario]}, Ds, device, epochs, lr)

    ind_D = np.argmin(np.array(errors))
    D = Ds[ind_D]

    create_irt_dataset(Y_bin_train, 'data/irt_dataset.jsonlines')

    train_irt_model(dataset_name='data/irt_dataset.jsonlines', 
                model_name=f'data/irt_model_{scenario}/', 
                D=D, lr=lr, epochs=epochs, device=device)
    
    return errors, errors2


Saving the training dataset in the needed format:

## Get the values for $\lambda$ which will be used to get the gp-IRT estimates (in conjunction with anchor methods)

In [19]:
def get_lambda(b, v):
    return (b**2)/(v+(b**2))

The variable `number_item` gives the number of data points we will sample per scenario:

In [20]:
def estimate_lambdas(err, Y_train, scenarios_position, lb_scenarios, number_item):
    lambds = {} 

    for i,scenario in enumerate(lb_scenarios.keys()):
        v = np.var(Y_train[:,scenarios_position[scenario]], axis=1).mean()
        b = np.mean(err[i]) 
        lambds[scenario] = get_lambda(b, v/(4*number_item))

    return lambds

In [32]:
def process_tb_data(Y_bin_train, Y_train, lb_scenarios, scenarios_position, scenario='lb'):
    errors, errors2 = train_IRT_for_scenario(Y_bin_train, Y_train, scenario, lb_scenarios)
    A, B, Theta = load_irt_parameters(f'data/irt_model_{scenario}/')
    ind_D = np.argmin(np.array(errors))

    for num_item in [10, 15, 20, 30, 50]:
        lambds = estimate_lambdas(errors2[ind_D], Y_train, scenarios_position, {scenario: lb_scenarios[scenario]}, num_item)
        
        # read lb_anchor_{num_item}.pickle file
        with open(f'data/lb_anchor_{num_item}.pickle', 'rb') as handle:
            anchor_data = pickle.load(handle) # contains 'seen_examples', 'examples_weights', 'scenarios_position', 'subscenarios_position'
        # save to tinybenchmark_lb file, putting 'seen_examples', 'examples_weights', 'irt_parameters', 'scenarios_position', 'subscenarios_position', 'optimal_lambdas'
        # collapse all anchors into a single list

        if scenario=='lb':
            with open(f'data/tinybenchmark_{scenario}_{num_item}.pickle', 'wb') as handle:
                pickle.dump({scenario: {'seen_examples': anchor_data['seen_examples'][scenario], 
                                    'examples_weights': anchor_data['examples_weights'], 
                                    'irt_parameters': {'A': A, 'B': B}, 
                                    'scenarios_position': anchor_data['scenarios_position'], 
                                    'subscenarios_position': anchor_data['subscenarios_position'], 
                                    'optimal_lambdas': lambds}
                                }, handle, protocol=pickle.HIGHEST_PROTOCOL)
        else:
            with open(f'data/tinybenchmark_{scenario}_{num_item}.pickle', 'wb') as handle:
                s = lb_scenarios[scenario][0]
                pickle.dump({scenario: {'seen_examples': anchor_data['seen_examples'][s] + anchor_data['subscenarios_position']['lb'][s][0], 
                                    'examples_weights': anchor_data['examples_weights'][s], 
                                    'irt_parameters': {'A': A, 'B': B}, 
                                    'scenarios_position': anchor_data['scenarios_position'], 
                                    'subscenarios_position': anchor_data['subscenarios_position'], 
                                    'optimal_lambdas': lambds}
                                }, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [33]:
lb_scenarios

{'lb': ['harness_truthfulqa_mc_0',
  'harness_gsm8k_5',
  'harness_arc_challenge_25',
  'harness_hellaswag_10'],
 'gsm8k': ['harness_gsm8k_5'],
 'arc': ['harness_arc_challenge_25'],
 'hellaswag': ['harness_hellaswag_10'],
 'truthfulqa': ['harness_truthfulqa_mc_0']}

In [34]:
for s in ['arc', 'gsm8k', 'hellaswag', 'truthfulqa', 'lb']:
    process_tb_data(Y_bin_train, Y_train, lb_scenarios, scenarios_position, scenario=s)

  0%|          | 0/1 [00:00<?, ?it/s]

[10:07:50] config: model_type='multidim_2pl' epochs=2000              cli.py:109
           priors='hierarchical' initializers=[] dims=2 lr=0.1                  
           lr_decay=0.9999 dropout=0.5 hidden=100 vocab_size=None               
           log_every=200 seed=42 deterministic=True                             
           data_path: data/irt_val_dataset.jsonlines                  cli.py:111
           output directory: data/irt_val_model/                      cli.py:112
[10:07:52] amortized: False                                       dataset.py:112
[10:08:06] Vocab size: None                                       training.py:90
[10:08:07] Training Model...                                          cli.py:116
[10:08:07] args: {'device': 'cuda', 'num_items': 26700,          training.py:134
           'num_subjects': 234}                                                 
           Parsed Model Args: {'device': 'cuda', 'num_items':    training.py:147
           26700, 'num_subje

100%|██████████| 1/1 [01:43<00:00, 103.27s/it]


[10:09:40] config: model_type='multidim_2pl' epochs=2000              cli.py:109
           priors='hierarchical' initializers=[] dims=2 lr=0.1                  
           lr_decay=0.9999 dropout=0.5 hidden=100 vocab_size=None               
           log_every=200 seed=42 deterministic=True                             
           data_path: data/irt_dataset.jsonlines                      cli.py:111
           output directory: data/irt_model_arc/                      cli.py:112
[10:09:42] amortized: False                                       dataset.py:112
[10:10:00] Vocab size: None                                       training.py:90
[10:10:02] Training Model...                                          cli.py:116
[10:10:02] args: {'device': 'cuda', 'num_items': 26700,          training.py:134
           'num_subjects': 293}                                                 
           Parsed Model Args: {'device': 'cuda', 'num_items':    training.py:147
           26700, 'num_subje

  0%|          | 0/1 [00:00<?, ?it/s]

[10:11:41] config: model_type='multidim_2pl' epochs=2000              cli.py:109
           priors='hierarchical' initializers=[] dims=2 lr=0.1                  
           lr_decay=0.9999 dropout=0.5 hidden=100 vocab_size=None               
           log_every=200 seed=42 deterministic=True                             
           data_path: data/irt_val_dataset.jsonlines                  cli.py:111
           output directory: data/irt_val_model/                      cli.py:112
[10:11:43] amortized: False                                       dataset.py:112
[10:11:57] Vocab size: None                                       training.py:90
[10:11:58] Training Model...                                          cli.py:116
[10:11:58] args: {'device': 'cuda', 'num_items': 26700,          training.py:134
           'num_subjects': 234}                                                 
           Parsed Model Args: {'device': 'cuda', 'num_items':    training.py:147
           26700, 'num_subje

100%|██████████| 1/1 [01:44<00:00, 104.36s/it]


[10:13:32] config: model_type='multidim_2pl' epochs=2000              cli.py:109
           priors='hierarchical' initializers=[] dims=2 lr=0.1                  
           lr_decay=0.9999 dropout=0.5 hidden=100 vocab_size=None               
           log_every=200 seed=42 deterministic=True                             
           data_path: data/irt_dataset.jsonlines                      cli.py:111
           output directory: data/irt_model_gsm8k/                    cli.py:112
[10:13:34] amortized: False                                       dataset.py:112
[10:13:53] Vocab size: None                                       training.py:90
[10:13:54] Training Model...                                          cli.py:116
[10:13:54] args: {'device': 'cuda', 'num_items': 26700,          training.py:134
           'num_subjects': 293}                                                 
           Parsed Model Args: {'device': 'cuda', 'num_items':    training.py:147
           26700, 'num_subje

: 

: 

In [26]:
# read lb_anchor_{num_item}.pickle file
with open(f'data/lb_anchor_30.pickle', 'rb') as handle:
    anchor_data = pickle.load(handle)

In [31]:
lb_scenarios['arc'][0]

'harness_arc_challenge_25'

In [ ]:
s = lb_scenarios['arc'][0]
anchor_data['subscenarios_position']['lb'][s][0]

In [29]:
anchor_data['examples_weights'][lb_scenarios['arc'][0]]

TypeError: unhashable type: 'list'

In [26]:
with open('data/tinybenchmark_lb_30.pickle', 'rb') as handle:
    data = pickle.load(handle)

data['lb']['scenarios_position']['truthfulqa']


[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,
